In [ ]:
# Import Pynq libraries
from pynq import Overlay
from pynq import allocate
import numpy as np

In [ ]:
# Load the overlay
overlay = Overlay('mult_stream_pynq.bit')

# Set up proxy variables to make it easier to access the DMA module and the IP (multiplier)
# so that we can communicate with it.
d = overlay.axi_dma_0
m = overlay.ip_inst

In [ ]:
# Set up the input and output buffers.  Use the `allocate` function since we need
# to be able to find the physical addresses of these arrays later for the DMA.
num_values = 400
in_buffer = allocate(shape=(num_values,), dtype=np.uint32)
out_buffer = allocate(shape=(num_values,), dtype=np.uint32)

# Load the array with values: this happens on the CPU.
for i in range(num_values):
    in_buffer[i] = i

In [ ]:
# Set the multiplication constant
m.register_map.constant_r = 13

In [ ]:
# Send and receive data using DMA.  This takes care of all the setup and transfer
# and automatically takes the length of transfer based on the size used during `allocate`.
d.sendchannel.transfer(in_buffer)
d.recvchannel.transfer(out_buffer)
d.sendchannel.wait()
d.recvchannel.wait()

In [ ]:
# Optional to check the status of the DMA module
d.sendchannel.idle, d.sendchannel.running, d.recvchannel.idle, d.recvchannel.running

In [ ]:
# Print the output for manual verification
out_buffer